In [8]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Sales Data Analysis").getOrCreate()

# File location and type
file_location = "/Users/inbravo/Documents/GitHub/python-feature-set/dataset/super-market/super-market-sales-less-record.csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ",") \
  .load(file_location)

display(df)
df.printSchema()
# display(df.schema.names)
# Iterate over the elements of the array column
for row in df.collect():
    print(row.Product_Line, row.Gross_Income)

DataFrame[Invoice_ID: string, Branch: string, City: string, Customer_Type: string, Gender: string, Product_Line: string, Unit_Price: double, Quantity: int, Tax_5%: double, Total: double, Date: string, Time: timestamp, Payment: string, cogs: double, Gross_Margin_Percentage: double, Gross_Income: double, Rating: double]

root
 |-- Invoice_ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer_Type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product_Line: string (nullable = true)
 |-- Unit_Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax_5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- Gross_Margin_Percentage: double (nullable = true)
 |-- Gross_Income: double (nullable = true)
 |-- Rating: double (nullable = true)

Health and beauty 26.1415
Electronic accessories 3.82
Home and lifestyle 16.2155
Health and beauty 2.0175
Home and lifestyle 48.69
Food and beverages 1.592
Home and lifestyle 3.291
Fashion accessories 30.919


In [9]:
from pyspark.sql.functions import round
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import IntegerType
   
# Typecast the 'Gross income' values to Integer type    
df = df.withColumn("Gross_Income", df["Gross_Income"].cast(IntegerType()))

# Round gross income to 2 decimal places
df = df.withColumn("Gross_Income", round(df["Gross_Income"], 2))

# Group by the "Product line"
df = df.groupBy("Product_Line")

# Sum of the gross income
gross_income_sum = _sum("Gross_Income")
print(gross_income_sum)

# Sum the gross income
df = df.agg(gross_income_sum).alias("Gross_Income")

Column<'sum(Gross_Income)'>


In [ ]:
from pyspark.sql.functions import round
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import IntegerType

grouped_df=df.groupBy("Product_line").agg(_sum("Gross_Income").alias("Total_Gross"))   #checking to see what type of products yeilds more income
df_round = grouped_df.withColumn("Rounded_Total_Sales", round(grouped_df["Total_Gross"], 2))
display(df_round)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Gross_Income` cannot be resolved. Did you mean one of the following? [`sum(Gross_Income)`, `Product_Line`].;
'Aggregate [Product_line#356], [Product_line#356, sum('Gross_Income) AS Total_Gross#467]
+- SubqueryAlias Gross_Income
   +- Aggregate [Product_Line#356], [Product_Line#356, sum(Gross_Income#421) AS sum(Gross_Income)#457L]
      +- Project [Invoice_ID#351, Branch#352, City#353, Customer_Type#354, Gender#355, Product_Line#356, Unit_Price#357, Quantity#358, Tax_5%#359, Total#360, Date#361, Time#362, Payment#363, cogs#364, Gross_Margin_Percentage#365, round(Gross_Income#402, 2) AS Gross_Income#421, Rating#367]
         +- Project [Invoice_ID#351, Branch#352, City#353, Customer_Type#354, Gender#355, Product_Line#356, Unit_Price#357, Quantity#358, Tax_5%#359, Total#360, Date#361, Time#362, Payment#363, cogs#364, Gross_Margin_Percentage#365, cast(Gross_Income#366 as int) AS Gross_Income#40

In [ ]:
# entire code in single place
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, hour, to_timestamp, sum, avg, dayofweek

# Initialize Spark Session (if not already created in Databricks)
spark = SparkSession.builder.appName("SupermarketSalesETL").getOrCreate()

# Load the dataset from DBFS
file_path = "/FileStore/tables/supermarket_sales___Sheet1-1.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

# 1️⃣ Data Cleaning
df = df.withColumn("Date", to_date(col("Date"), "M/d/yyyy"))
df = df.withColumn("Time", to_timestamp(col("Time"), "HH:mm"))
df = df.withColumn("Hour", hour(col("Time")))
df = df.withColumn("Day_of_Week", dayofweek(col("Date")))

# 2️⃣ Feature Engineering & Aggregations

# Total Sales Per Branch
df_branch_sales = df.groupBy("Branch").agg(sum("Total").alias("Total_Sales"))

# Average Basket Size (Avg Total per Transaction)
df_avg_basket = df.agg(avg("Total").alias("Avg_Basket_Size"))

# Total Spending Per Customer Type & Gender
df_customer_spend = df.groupBy("Customer type", "Gender").agg(sum("Total").alias("Total_Spending"))

# Preferred Payment Methods
df_payment_method = df.groupBy("Payment").agg(sum("Total").alias("Total_Spent"))

# Peak Sales Hours
df_peak_hours = df.groupBy("Hour").agg(sum("Total").alias("Total_Sales")).orderBy(col("Total_Sales").desc())

# Busiest Days of the Week
df_busy_days = df.groupBy("Day_of_Week").agg(sum("Total").alias("Total_Sales")).orderBy(col("Total_Sales").desc())

# Total Revenue & Profit per Branch
df_branch_financials = df.groupBy("Branch").agg(
    sum("Total").alias("Total_Revenue"),
    sum("gross income").alias("Total_Profit")
)

# Average Rating Per Product Line
df_avg_rating = df.groupBy("Product line").agg(avg("Rating").alias("Avg_Rating"))

# Most Frequently Purchased Products
df_top_products = df.groupBy("Product line").agg(sum("Quantity").alias("Total_Quantity")).orderBy(col("Total_Quantity").desc())

# Showing Results, We can do the same with display for visualization
df_branch_sales.show()
df_avg_basket.show()
df_customer_spend.show()
df_payment_method.show()
df_peak_hours.show()
df_busy_days.show()
df_branch_financials.show()
df_avg_rating.show()
df_top_products.show()